In [ ]:
df_dictionary = pd.read_csv('../data/utils/DF_DICTIONARY.csv')


## DataFrame Dictionary Functions

In [ ]:
def get_indicator_name(indicator_code):
    return df_dictionary.loc[df_dictionary['Indicator Code'] == indicator_code, 'Indicator Name'].iloc[0]

In [ ]:
def get_indicator_names(indicator_codes):
    response = []
    for index_code in indicator_codes:
        if(index_code.str.lower() != 'year'):
            response.append(get_indicator_name(index_code))
    return response

## DataFrame filtering by missing values count

In [ ]:
def filter_by_cols_first(df, percentage):
    cols_filtered_first = df.dropna(thresh=int(df.shape[0] * percentage), axis=1)
    return cols_filtered_first.dropna(thresh=int(cols_filtered_first.shape[1] * percentage), axis=0)

In [ ]:
def filter_by_rows_first(df, percentage):
    rows_filtered_second = df.dropna(thresh=int(df.shape[1] * percentage), axis=0)
    return rows_filtered_second.dropna(thresh=int(rows_filtered_second.shape[0] * percentage), axis=1)


In [ ]:
#TODO: Check this alternative function
# Calculate the percentage of non-missing values for each column
# threshold = 0.7  # Keep indicators with at least 70% non-missing data
# non_missing_percentage = data.notnull().mean()

# # Filter indicators
# useful_indicators = non_missing_percentage[non_missing_percentage >= threshold].index
# filtered_data = data[useful_indicators]

In [ ]:
def add_indicators_of_year(df, year):
    new_year_indicator = arg_di_df[arg_di_df["Year"] == year]
    new_year_indicator.missing.missing_case_summary()
    df = pd.concat([df, new_year_indicator]).sort_index()

## Old functions

#### Getting the percentage of variables with a minimum percentage of missing values

In [ ]:
def percentage_of_variables_with_pct_missing(df: pd.DataFrame, percentage: float, greater_than_pct = True):
    variable_summary = df.missing.missing_variable_summary()
    if greater_than_pct:
        return round(variable_summary[variable_summary['pct_missing'] >= percentage]['variable'].count() / variable_summary.shape[0], 2)
    else:
        return round(variable_summary[variable_summary['pct_missing'] <= percentage]['variable'].count() / variable_summary.shape[0], 2)

#### Getting the column names with a maximum percentage of missing values

In [ ]:
def columns_with_pct_missing(df: pd.DataFrame, percentage: float, greater_than_pct = True):
    variable_summary = df.missing.missing_variable_summary()
    if greater_than_pct:
        missing_variables_code = list(variable_summary[variable_summary['pct_missing'] >= percentage]['variable'])
    else:
        missing_variables_code = list(variable_summary[variable_summary['pct_missing'] <= percentage]['variable'])
    return missing_variables_code

### Getting the rows with more missing values

In [ ]:
def cases_with_pct_missing(df: pd.DataFrame, percentage: float, greater_than_pct = True):
    variable_summary = df.missing.missing_case_summary()
    if greater_than_pct:
        indexes_of_cases = list(variable_summary[variable_summary['pct_missing'] > percentage]['case'])
    else:
        indexes_of_cases = list(variable_summary[variable_summary['pct_missing'] < percentage]['case'])
    return arg_di_df.iloc[indexes_of_cases]